In [3]:
import os
import torch
from torch import nn
import dltools

In [ ]:
def read_imdb(data_dir,is_train):
    data,labels = [],[]
    for label in ('pos','neg'):
        folder_name = os.path.join(data_dir,'train' if is_train else 'test',label)
        for file in os.listdir(folder_name):
            with open(os.path.join(folder_name,file),'rb') as f:
                review = f.read().decode('utf-8').replace('\n','')
                data.append(review)
                labels.append(1 if label == 'pos' else 0)
    return data,labels

data_dir = "./aclimdb"
train_data = read_imdb(data_dir,is_train=True)
print('train_len:',len(train_data)[0])
for x ,y in zip(train_data[0][:3],train_data[1][3]):
    print('label:',y,"review:",x[:60])



In [ ]:
train_token = dltools.token(train_data[0],token='word')
vocab = dltools.Vocab(train_token,min_freq=5,reserved_token=["<pad>"])

In [ ]:
dltools.set_figsize()
dltools.plt.xlabel('# tokens perview')
dltools.plt.ylabel('count')
dltools.plt.hist([len(line) for line in train_tokens],bins=range(0,1000,50))

In [ ]:
num_steps = 500
train_feature = torch.tensor([dltools.truncate_pad(vocab[line],num_steps,vocab['<pad>']) for line in train_token])
print(train_feature.shape)

In [ ]:
train_iter = dltools.load_array((train_feature,torch.tensor(train_data[1])),64)

for x ,y in train_iter:
    print("X:",x.shape,",y:",y.shape)
    break
print('小批量数目：',len(train_iter))

In [ ]:
def load_data_imdb(batch_size,num_steps=500):
    data_dir = dltools.download_extract("aclImdb","aclImdb")
    train_data = read_imdb(data_dir,True)
    test_data = read_imdb(data_dir,False)
    train_tokens = dltools.tokenize(train_data[0],token='word')
    test_tokens = dltools.tokenize(test_data[0],token='word')
    vocab = dltools.Vocab(train_tokens,min_freq=5)
    train_features = torch.tensor([dltools.truincate_pad(vocab[line],num_steps,vocab["<pad>"]) for line in train_tokens])
    test_features = torch.tensor([dltools.truincate_pad(vocab[line],num_steps,vocab["<pad>"]) for line in test_tokens])
    train_iter = dltools.load_array((train_features,torch.tensor(train_data[1])),batch_size)
    test_iter = dltools.load_array((test_features,torch.tensor(train_data[1])),batch_size,is_train=False)
    return train_iter,test_iter,vocab